# IMPORTING LIBRARIES

In [ ]:
import numpy as np #Biblioteca de suporte de arrays e matriz multidimensionais
import pandas as pd #Biblioteca de manipulação e análise de dado
import os #Esse modulo prove alguns metodos para atuar com o sistema operacional, nesse projeto utilizado para gerenciamento de diretorios
import sys #Esse modulo prove acesso à algumas variaveis de sistema, nesse projeto utilizado para gerenciamento de diretorios
import matplotlib.pyplot as plt #Biblioteca padrão pala plotagem de graficos
import seaborn as sns #Biblioteca para melhoria de interface dos graficos
sys.path.append('../abstract_normalizer')
import dictionaries
import json
import operator
from matplotlib.ticker import (MultipleLocator, FormatStrFormatter, AutoMinorLocator)
import math

## Function that removes files that have especified extensions from a list of files 'li'

In [3]:
def cleanDirectory(li):
    try:
        for directory in li:
            if(not directory.endswith(".json") ):
                li.remove(directory)
                cleanDirectory(li)
    except:
        print("")

## Function that gets all the fields, disciplines and journalsof the corpus

In [4]:
def getAllFields():
    return {
    "AGRONOMY", 
    "BUSINESS", 
    "CHEMISTRY", 
    "COGNITIVE LINGUISTICS",
    "COMPUTER SCIENCE",
    "ENGINEERING", 
    "HEALTH CARE SCIENCES & SERVICES", 
    "LINGUISTICS", 
    "PHYSICS", 
    "TELECOMMUNICATIONS"
    }

def getAllDisciplines():
    return {
            "BIOLOGICAL SCIENCES",
            "SOCIAL & HUMAN SCIENCES",
            "HARD SCIENCES"
    }

def getAllJournals():
    return {
    "ACADEMIC MEDICINE",
    "ACADEMY OF MANAGEMENT ANNALS" ,
    "ACADEMY OF MANAGEMENT REVIEW" ,
    "ADVANCED MATERIALS" ,
    "ADVANCES IN AGRONOMY, VOL 129",
    "ADVANCES IN AGRONOMY, VOL 130",
    "ADVANCES IN AGRONOMY, VOL 131",
    "ADVANCES IN AGRONOMY, VOL 132",
    "ADVANCES IN AGRONOMY, VOL 133",
    "ADVANCES IN AGRONOMY, VOL 134",
    "ADVANCES IN AGRONOMY, VOL 135",
    "ADVANCES IN AGRONOMY, VOL 136",
    "ADVANCES IN AGRONOMY, VOL 137",
    "ADVANCES IN AGRONOMY, VOL 138",
    "ADVANCES IN AGRONOMY, VOL 139",
    "AGRICULTURAL AND FOREST METEOROLOGY",
    "AGRONOMY FOR SUSTAINABLE DEVELOPMENT",
    "ANNUAL REVIEW OF APPLIED LINGUISTICS",
    "APPLIED LINGUISTICS",
    "ARCHIVES OF COMPUTATIONAL METHODS IN ENGINEERING",
    "BMJ QUALITY & SAFETY",
    "BRAIN AND LANGUAGE",
    "CHEMICAL REVIEWS" ,
    "CHEMICAL SOCIETY REVIEWS" ,
    "COGNITIVE LINGUISTICS" ,
    "COMBUSTION AND FLAME",
    "COMPOSITES PART B ENGINEERING",
    "COMPUTER AIDED CIVIL AND INFRASTRUCTURE ENGINEERING",
    "COMPUTER METHODS IN APPLIED MECHANICS AND ENGINEERING",
    "ENERGY & ENVIRONMENTAL SCIENCE" ,
    "GLOBAL CHANGE BIOLOGY BIOENERGY",
    "HEALTH AFFAIRS",
    "IEEE COMMUNICATIONS MAGAZINE",
    "IEEE COMMUNICATIONS SURVEYS AND TUTORIALS",
    "IEEE JOURNAL ON SELECTED AREAS IN COMMUNICATIONS",
    "IEEE NETWORK",
    "IEEE TRANSACTIONS ON INDUSTRIAL INFORMATICS",
    "IEEE TRANSACTIONS ON MEDICAL IMAGING",
    "IEEE WIRELESS COMMUNICATIONS",
    "INTERNATIONAL JOURNAL OF ENGINEERING SCIENCE",
    "JOURNAL OF MANAGEMENT" ,
    "JOURNAL OF MARKETING" ,
    "JOURNAL OF MEMORY AND LANGUAGE",
    "JOURNAL OF SECOND LANGUAGE WRITING",
    "JOURNAL OF STATISTICAL SOFTWARE",
    "JOURNAL OF THE ACADEMY OF MARKETING SCIENCE" ,
    "MEDICAL IMAGE ANALYSIS",
    "MILBANK QUARTERLY",
    "MODERN LANGUAGE JOURNAL",
    "NATURE CHEMISTRY" ,
    "NATURE PHYSICS" ,
    "PHYSICAL REVIEW X" ,
    "PHYSICS REPORTS REVIEW SECTION OF PHYSICS LETTERS" ,
    "QUALITY & SAFETY" ,
    "REPORTS ON PROGRESS IN PHYSICS" ,
    "REVIEW OF COGNITIVE LINGUISTICS" ,
    "REVIEWS OF MODERN PHYSICS" ,
    "THEORETICAL AND APPLIED GENETICS",
    "VALUE IN HEALTH"
    }

## Function that gets the Title of the abstract

In [5]:
def getTitle(title):
    txtFiles = list()
    for (dirpath, dirnames, filenames) in os.walk("../raw"):
        txtFiles += [os.path.join(dirpath, file) for file in filenames]

    for txtFile in txtFiles:
        if txtFile.endswith(".txt"):
            for line in list(open(txtFile)):
                if(line.startswith("TI ")):
                    nextline = list(open(txtFile))[list(open(txtFile)).index(line)+1];
                    if(not nextline.startswith("SO ")):
                        fileTitle = line.replace("TI ","").replace("\n"," ").replace("  "," ")+" ".join(nextline.split())
                    else:
                        fileTitle = line.replace("TI ","").replace("\n"," ")
                    if(fileTitle == title):
                        return fileTitle

##  Functions that get the Journal based on a file name

In [6]:
def searchJournal(file1,file2):
    retorno = ""
    for j in getAllJournals():
        if(file1.find(j) != -1):
            retorno = file1
        elif(file2.find(j) != -1):
            retorno = file2
        elif( (file2.find(j) != -1) and (file1.find(j) != -1) ):
            retorno = "False"
    return retorno

def getJournal(file):
    file = file.split('.seg.json')[0]
    file1 = file.split('_')[-3].replace('-',' ').upper()
    file2 = file.split('_')[-2].replace('-',' ').upper()
    journal = searchJournal(file1,file2)
    return journal

## Function that gets Abstract's info (journal, field, discipline) from a json file 'file'

In [16]:
def getInfoFromFile(file):
            fields = dictionaries.dictionaryFields()
            disciplines = dictionaries.dictionaryDisciplines()
            journals = getAllJournals() 
            journal = ""
            
            journal = getJournal(file)    
            
            field = fields.get(journal)
            discipline = disciplines.get(field)
            abstract = dict()
            abstract['journal'] = journal
            abstract['field']   = field
            abstract['discipline'] = discipline
            abstract['all'] = 'all'
            return abstract

## Function that orders a dict with index:values by values

In [8]:
def sortDict(d):
    lista  = (sorted(d.items(), key = lambda x : x[1], reverse=True))
    retorno = dict()
    for value in lista:
        retorno[value[0]] = value[1]
    return retorno

## Function that receives a dict with indexes:values and returns a dict with only the 30 heighest values in the parameter dict

In [9]:
def first30(pdict):
    d = pdict.copy()
    labels = list(d.keys())
    counter = 0
    for l in labels:
        if(counter>=30):
            d.pop(l)
        counter += 1
    return d

## Function that plots charts

In [10]:
def generateChart(d, opt, info, optdir):
    
    dic = d
    
    #selecting only the 30 frames with more occurrence
    dic = first30(dic)
    
    #setting the data that will appear in the chart
    labels = list(dic.keys())
    values = list(dic.values())
    index = np.arange(len(labels))
    
    #used to allow xaxis locators settings
    fig, ax = plt.subplots()
    
    #plotting bar chart
    ax.barh(index, values)
    
    #setting labels
    if(optdir == 'frame'):
        plt.ylabel('Frames', fontsize=5)
    elif(optdir == 'lexical_unit'):
        plt.ylabel('Lexical Units', fontsize=5)

    plt.xlabel('Occurrences', fontsize=5)
    
    plt.yticks(index, labels, fontsize=5, rotation=0)
    
    major = 10
    minor = 1
    bigger = max(values)
    if(bigger > 10):
        major = bigger/10
        major = math.ceil(major)
        while(major % 10 != 0):
            major = major+1
        minor = major/10
    else:
        major = 2
        minor = 1
    
    #configuring xtick Major and minor locators
    mlocatormax = MultipleLocator(major)
    mlocatormin = MultipleLocator(minor)
    
    ax.xaxis.set_major_locator(mlocatormax)
    ax.xaxis.set_major_formatter(FormatStrFormatter('%d'))
    ax.xaxis.set_minor_locator(mlocatormin)
    ax.xaxis.set_view_interval(0,bigger+major)
    
    #setting locators size
    ax.tick_params(axis='x', which='major', labelsize=6)
    ax.tick_params(axis='y', which='major', labelsize=5)
    
    if(info == 'all'):
        info = 'All Corpus'
        
    plt.title(info)
    
    if(optdir == 'frame'):
        plt.savefig('export/'+opt+'/frame/'+info+'.png', bbox_inches='tight', dpi=300)
    elif(optdir == 'lexical_unit'):
        plt.savefig('export/'+opt+'/lexical_unit/'+info+'.png', bbox_inches='tight', dpi=300)
        
    #plt.show()
    plt.close()

## Function that creates the directories where charts will be saved

In [11]:
def createDirectories():
    if not os.path.exists('export'):
        os.mkdir('export')

    if not os.path.exists('export/field'):
        os.mkdir('export/field')

    if not os.path.exists('export/discipline'):
        os.mkdir('export/discipline')

    if not os.path.exists('export/journal'):
        os.mkdir('export/journal')

    if not os.path.exists('export/all'):
        os.mkdir('export/all')
        
    if not os.path.exists('export/field/frame'):
        os.mkdir('export/field/frame')

    if not os.path.exists('export/discipline/frame'):
        os.mkdir('export/discipline/frame')

    if not os.path.exists('export/journal/frame'):
        os.mkdir('export/journal/frame')

    if not os.path.exists('export/all/frame'):
        os.mkdir('export/all/frame')

    if not os.path.exists('export/field/lexical_unit'):
        os.mkdir('export/field/lexical_unit')

    if not os.path.exists('export/discipline/lexical_unit'):
        os.mkdir('export/discipline/lexical_unit')

    if not os.path.exists('export/journal/lexical_unit'):
        os.mkdir('export/journal/lexical_unit')

    if not os.path.exists('export/all/lexical_unit'):
        os.mkdir('export/all/lexical_unit')
        
createDirectories()

## Function that receives a 

In [12]:
def framesPerCategory(categories, files, opt, path):
    mensagem = "Dados de: \n"
    exc_data = dict()
    if(opt == 'all'):
        categories = list([opt])
    for cat in categories:
            dchart = dict()
            #dexcel = dict()
            for file_dir in files:
                abstract = getInfoFromFile(path+'/'+file_dir)
                if(abstract.get(opt) == cat):
                    file = open(path+'/'+file_dir,"r").readlines()
                    for line in file:
                        jsonline = json.loads(line)
                        for frame in jsonline.get('frames'):
                            frameNom = frame.get('target').get('name')
                            if(dchart.get(frameNom) != None):
                                dchart[frameNom] += 1
                            else:
                                dchart[frameNom] = 1
            dchart = sortDict(dchart)
            if(dchart):
                exc_data[cat] = dchart.copy()
                generateChart(dchart, opt, cat, 'frame')
            else:
                mensagem = mensagem+cat+"\n"
    mensagem = mensagem+"não encontrados"
    if(mensagem != "Dados de: \n"+"não encontrados"):
        print(mensagem)
    return exc_data

In [13]:
def fElementsPerCategory(categories, files, opt, path):
    mensagem = "Dados de: \n"
    exc_data = dict()
    if(opt == 'all'):
        categories = list([opt])
    for cat in categories:
            dchart = dict()
            #dexcel = dict()
            for file_dir in files:
                abstract = getInfoFromFile(path+'/'+file_dir)
                if(abstract.get(opt) == cat):
                    file = open(path+'/'+file_dir,"r").readlines()
                    for line in file:
                        jsonline = json.loads(line)
                        for frame in jsonline.get('frames'):
                            for span in frame.get('target').get('spans'):
                                    LUnom = span.get('text')
                                    if(dchart.get(LUnom) != None):
                                        dchart[LUnom] += 1
                                    else:
                                        dchart[LUnom] = 1
#                             for anno in frame.get('annotationSets'):
#                                 for felement in anno.get('frameElements'):
#                                     elementnom = felement.get('name')
#                                     if(dchart.get(elementnom) != None):
#                                         dchart[elementnom] += 1
#                                     else:
#                                         dchart[elementnom] = 1
            dchart = sortDict(dchart)
            if(dchart):
                exc_data[cat] = dchart.copy()
                generateChart(dchart, opt, cat+" (Lexical Units)", 'lexical_unit')
            else:
                mensagem = mensagem+cat+"\n"
    mensagem = mensagem+"não encontrados"
    if(mensagem != "Dados de: \n"+"não encontrados"):
        print(mensagem)
    return exc_data

In [14]:
def generateExcel(fpcat, opt, optdir):
    if(opt != "all"):
        for v in fpcat:
            vDict = fpcat.get(v)
            vDF = pd.DataFrame({optdir+'s':list(vDict.keys()),'occurrence':list(vDict.values())})
            vDF.to_excel("export/"+opt+"/"+optdir+"/"+v+".xlsx", index=False)
    else:
        fpcat = pd.DataFrame({optdir+'s':list(fpcat.get('all').keys()),'occurrence':list(fpcat.get('all').values())})
        fpcat.to_excel("export/"+opt+"/"+optdir+"/All_Abstracts.xlsx", index=False)

## selecting only the files we need

In [15]:
path = "../semafor_output" #./semafor_output
files = os.listdir(path)
cleanDirectory(files) 
a = dictionaryFields()

NameError: name 'dictionaryFields' is not defined

In [24]:
#sorting fields alphabetically
fields = list(getAllFields())
fields.sort()

#sorting disciplines alphabetically
disciplines = list(getAllDisciplines())
disciplines.sort()

#sorting journals alphabetically
journals = list(getAllJournals())
journals.sort()

#creating the directories needed
createDirectories()

# bar charts for all abstracts, by field, by discipline and by journal
fpall = framesPerCategory(files,files,'all',path)
fpfields = framesPerCategory(fields,files,'field',path)
fpdisciplines = framesPerCategory(disciplines,files,'discipline',path)
fpjournal = framesPerCategory(journals,files,'journal',path)

#FEpall = fElementsPerCategory(files,files,'all',path)
FEpfields = fElementsPerCategory(fields,files,'field',path)
FEpdisciplines = fElementsPerCategory(disciplines,files,'discipline',path)
FEpjournals = fElementsPerCategory(journals,files,'journal',path)

#xls files for all abstracts, by field, by discipline and by journal
#generateExcel(fpall,"all",'frame')
generateExcel(fpfields,"field",'frame')
generateExcel(fpdisciplines,"discipline",'frame')
generateExcel(fpjournal,"journal",'frame')

#xls files for all abstracts, by field, by discipline and by journal (frame elements)
#generateExcel(FEpall,"all",'lexical_unit')
generateExcel(FEpfields,"field",'lexical_unit')
generateExcel(FEpdisciplines,"discipline",'lexical_unit')
generateExcel(FEpjournals,"journal",'lexical_unit')


Dados de: 
ADVANCES IN AGRONOMY, VOL 136
QUALITY & SAFETY
não encontrados
Dados de: 
ADVANCES IN AGRONOMY, VOL 136
QUALITY & SAFETY
não encontrados


In [16]:
#quantifica os frames que aparecem em cada categoria, maximo 3 (que é o que queremos)
def countFramesInCat(fpcat,indict):
    for item in fpcat:
        for f in first30(fpcat.get(item).copy()):
            if(indict.get(f) != None):
                indict[f] += 1
            else:
                indict[f] = 1 
    return indict

In [17]:
#remove os frames que apareceram em menos que três(todas) disciplines
def getinfirst30(indict,limit):
    keys = list(indict.keys())
    for key in keys:
        if(indict.get(key) < limit):
            indict.pop(key)
    return indict

In [18]:
#somando as ocorrencias dos frames que estao entre os 30 em todos, em cada um dos items de categoria
def suminfirst30occurrences(indict, fpcatdict, out):
    for oc in indict.copy():
        for u in fpcatdict.copy():
            for p in fpcatdict.get(u).copy():
                if(p == oc):
                    if(out.get(oc) != None):
                        out[oc] = out[p]+fpcatdict.get(u).copy().get(p)
                    else:
                        out[oc] = fpcatdict.get(u).copy().get(p)
    return out

In [19]:
inall = dict()
indisciplines = dict()
infields = dict()
injournals = dict()

inFEall = dict()
inFEdisciplines = dict()
inFEfields = dict()
inFEjournals = dict()

indisciplines = countFramesInCat(fpdisciplines,indisciplines)
infields = countFramesInCat(fpfields,infields)
injournals = countFramesInCat(fpjournal,injournals)

#test frame LU
inFEdisciplines = countFramesInCat(FEpdisciplines,inFEdisciplines)
inFEfields = countFramesInCat(FEpfields,inFEfields)
inFEjournals = countFramesInCat(FEpjournals,inFEjournals)
            
#sorting
inall = sortDict(inall)

infields = sortDict(infields)
indisciplines = sortDict(indisciplines)
injournals = sortDict(injournals)

inFEfields = sortDict(inFEfields)
inFEdisciplines = sortDict(inFEdisciplines)
inFEjournals = sortDict(inFEjournals)

indisciplines = getinfirst30(indisciplines,3)
infields = getinfirst30(infields,10)
#excecao aqui, mais da metade, nao todos
injournals = getinfirst30(injournals,29)

inFEdisciplines = getinfirst30(inFEdisciplines,3)
inFEfields = getinfirst30(inFEfields,10)
#excecao aqui, mais da metade, nao todos
inFEjournals = getinfirst30(inFEjournals,29)


sumin30disciplines = dict()
sumin30fields = dict()
sumin30journals = dict()

suminFE30disciplines = dict()
suminFE30fields = dict()
suminFE30journals = dict()

#soma o valor das ocorrencias dos que aparecem em todas (que ja foram selecionados)
sumin30disciplines = suminfirst30occurrences(indisciplines, fpdisciplines, sumin30disciplines)
sumin30fields = suminfirst30occurrences(infields, fpfields, sumin30fields)
sumin30journals = suminfirst30occurrences(injournals, fpjournal, sumin30journals)

#sorting
sumin30disciplines = sortDict(sumin30disciplines)
sumin30fields = sortDict(sumin30fields)
sumin30journals = sortDict(sumin30journals)

#plotando
generateChart(sumin30disciplines,'discipline','Occurrence of frames in the 30 with more occurrence for all Disciplines','frame')
sumin30disciplinesDF = pd.DataFrame({'Frames':list(sumin30disciplines.keys()),'Occurrence':list(sumin30disciplines.values())})
sumin30disciplinesDF.to_excel("export/discipline/frame/"+"summary"+".xlsx", index=False)
                         
generateChart(sumin30fields,'field','Occurrence of frames in the 30 with more occurrence for all Fields','frame')
sumin30fieldsDF = pd.DataFrame({'Frames':list(sumin30fields.keys()),'Occurrence':list(sumin30fields.values())})
sumin30fieldsDF.to_excel("export/field/frame/"+"summary"+".xlsx", index=False)
        
generateChart(sumin30journals,'journal','Occurrence of frames in the 30 with more occurrence for more than half of journals','frame')
sumin30journalsDF = pd.DataFrame({'Frames':list(sumin30journals.keys()),'Occurrence':list(sumin30journals.values())})
sumin30journalsDF.to_excel("export/journal/frame/"+"summary"+".xlsx", index=False)
        
# #excecao
generateChart(injournals,'journal','Number of journals for frames in the 30 with more occurrence for more than half of journals','frame')
sumin30journalsDF = pd.DataFrame({'Frames':list(sumin30journals.keys()),'Occurrence':list(sumin30journals.values())})
sumin30journalsDF.to_excel("export/journal/frame/"+"summary_counting"+".xlsx", index=False)

#soma o valor das ocorrencias dos que aparecem em todas (que ja foram selecionados)
suminFE30disciplines = suminfirst30occurrences(inFEdisciplines, FEpdisciplines, suminFE30disciplines)
suminFE30fields = suminfirst30occurrences(inFEfields, FEpfields, suminFE30fields)
suminFE30journals = suminfirst30occurrences(inFEjournals, FEpjournals, suminFE30journals)

#sorting
suminFE30disciplines = sortDict(suminFE30disciplines)
suminFE30fields = sortDict(suminFE30fields)
suminFE30journals = sortDict(suminFE30journals)

#plotando
if(suminFE30disciplines):
    generateChart(suminFE30disciplines,'discipline','Occurrence of Lexical Units in the 30 with more occurrence for all Disciplines','lexical_unit')
    suminFE30disciplinesDF = pd.DataFrame({'Lexical Units':list(suminFE30disciplines.keys()),'Occurrence':list(suminFE30disciplines.values())})
    suminFE30disciplinesDF.to_excel("export/discipline/lexical_unit/"+"summary"+".xlsx", index=False)
                  
if(suminFE30fields):
    generateChart(suminFE30fields,'field','Occurrence of Lexical Units in the 30 with more occurrence for all Fields','lexical_unit')
    suminFE30fieldsDF = pd.DataFrame({'Lexical Units':list(suminFE30fields.keys()),'Occurrence':list(suminFE30fields.values())})
    suminFE30fieldsDF.to_excel("export/field/lexical_unit/"+"summary"+".xlsx", index=False)
        
if(suminFE30journals):
    generateChart(suminFE30journals,'journal','Occurrence of Lexical Units in the 30 with more occurrence for more than half of journals','lexical_unit')
    suminFE30journalsDF = pd.DataFrame({'Lexical Units':list(suminFE30journals.keys()),'Occurrence':list(suminFE30journals.values())})
    suminFE30journalsDF.to_excel("export/journal/lexical_unit/"+"summary"+".xlsx", index=False)
        
# #excecao
if(inFEjournals):
    generateChart(inFEjournals,'journal','Number of journals for Lexical Units in the 30 with more occurrence for more than half of journals','lexical_unit')
    suminFE30journalsDF = pd.DataFrame({'Lexical Units':list(suminFE30journals.keys()),'Occurrence':list(suminFE30journals.values())})
    suminFE30journalsDF.to_excel("export/journal/lexical_unit/"+"summary_counting"+".xlsx", index=False)
            
            

In [22]:
import numpy as np #Biblioteca de suporte de arrays e matriz multidimensionais
import pandas as pd #Biblioteca de manipulação e análise de dado
import os #Esse modulo prove alguns metodos para atuar com o sistema operacional, nesse projeto utilizado para gerenciamento de diretorios
import sys #Esse modulo prove acesso à algumas variaveis de sistema, nesse projeto utilizado para gerenciamento de diretorios
import matplotlib.pyplot as plt #Biblioteca padrão pala plotagem de graficos
import seaborn as sns #Biblioteca para melhoria de interface dos graficos
sys.path.append('../abstract_normalizer')
import dictionaries
import json
import operator
from matplotlib.ticker import (MultipleLocator, FormatStrFormatter, AutoMinorLocator)
import math

path = "../semafor_output" #./semafor_output
files = os.listdir(path)
cleanDirectory(files)

dict_frames = dict()
for file_dir in files:
    abstract = getInfoFromFile(path+'/'+file_dir)
    file = open(path+'/'+file_dir,"r").readlines()
    for line in file:
        jsonline = json.loads(line)
        for frame in jsonline.get('frames'):
            if(frame.get('target').get('name') == 'Scrutinity' or frame.get('target').get('name') == 'Quantity'): 
                
                if(dict_frames.get(frame.get('target').get('name')) == None):
                    dict_frames[frame.get('target').get('name')] = dict()
                    
                for span in frame.get('target').get('spans') :
                    lutext= span.get('text')
                    #if(dict_frames.get(frame.get('target').get('name')).get(lutext) != None):
                        #lus[frame.get('target').get('name')][lutext] += 1
                    #else:
                        #lus[frame.get('target').get('name')][lutext] = 1
                            
dict_frames = sortDict(dict_frames)
print(dict_frames)

#sumin30journalsDF = pd.DataFrame({'lexical_unit':list(lus.keys()), 'occurrences':list(lus.values())})
#.to_excel("export/"+"Cardinal_numbers_biological_sciences"+".xlsx", index=False)

{'Quantity': {}}


In [18]:
teste = dict()
teste['bbb'] = 'sla'
a = teste.get('bbb')
a

'sla'